In [1]:
%use adventOfCode

In [2]:
val aoc = AocClient.fromEnv().interactiveDay(2025, 8)
aoc.viewPartOne()

--- Day 8: Playground --- 
 Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad. 
 You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground! 
 Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical junction boxes . 
 Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes. 
 The Elves are trying to figure out which junction boxes to connect so that electricity can reach every junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input). 
 For example: 
 162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
 
 This list describes the position of 20 junction boxes, one per line. Each position is given as X,Y,Z coordinates. So, the first junction box in the list is at X=162 , Y=817 , Z=812 . 
 To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are as close together as possible according to straight-line distance . In this example, the two junction boxes which are closest together are 162,817,812 and 425,690,689 . 
 By connecting these two junction boxes together, because electricity can flow between them, they become part of the same circuit . After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits. 
 Now, the two junction boxes which are closest together but aren't already directly connected are 162,817,812 and 431,825,988 . After connecting them, since 162,817,812 is already connected to another junction box, there is now a single circuit which contains three junction boxes and an additional 17 circuits which contain one junction box each. 
 The next two junction boxes to connect are 906,360,560 and 805,96,715 . After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each. 
 The next two junction boxes are 431,825,988 and 425,690,689 . Because these two junction boxes were already in the same circuit , nothing happens! 
 This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be. 
 After making the ten shortest connections, there are 11 circuits: one circuit which contains 5 junction boxes, one circuit which contains 4 junction boxes, two circuits which contain 2 junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces 40 . 
 Your list contains many junction boxes; connect together the 1000 pairs of junction boxes which are closest together. Afterward, what do you get if you multiply together the sizes of the three largest circuits?

In [3]:
data class JunctionBox(val x: Int, val y: Int, val z: Int) {
    override fun toString(): String = "($x, $y, $z)"
}

fun JunctionBox.distanceTo(other: JunctionBox) =
    sqrt(
        (x - other.x).toDouble().pow(2) + (y - other.y).toDouble().pow(2) + (z - other.z).toDouble().pow(2),
    )

operator fun JunctionBox.minus(other: JunctionBox) = this.distanceTo(other)

fun <T> List<T>.carthesianProduct(): Sequence<Pair<T, T>> =
    indices.flatMap { a ->
        indices.mapNotNull { b ->
            if (a == b) null else minOf(a, b) to maxOf(a, b)
        }
    }.distinct()
        .asSequence()
        .map { (a, b) -> get(a) to get(b) }

class Playground(val junctionBoxes: List<JunctionBox>) {

    val carthesianProductByDistance = junctionBoxes.carthesianProduct().sortedBy { (a, b) -> a - b }

    /** Represents in which circuit ([Map.Entry.value]) each [JunctionBox] ([Map.Entry.key]) belongs to. */
    val junctionBoxToCircuit = mutableMapOf<JunctionBox, Int>()
    val circuits = mutableMapOf<Int, MutableList<JunctionBox>>()

    val circuitCount
        get() = circuits.size + junctionBoxes.count { it !in junctionBoxToCircuit }

    fun connect(junctionBoxes: Pair<JunctionBox, JunctionBox>): Int {
        val (a, b) = junctionBoxes
        val aPreset = a in junctionBoxToCircuit
        val bPreset = b in junctionBoxToCircuit

        return when {
            !aPreset && !bPreset -> {
                val key = (circuits.keys.maxOrNull() ?: -1) + 1
                junctionBoxToCircuit[a] = key
                junctionBoxToCircuit[b] = key
                circuits[key] = mutableListOf(a, b)
                key
            }

            aPreset && !bPreset -> {
                val key = junctionBoxToCircuit[a]!!
                junctionBoxToCircuit[b] = junctionBoxToCircuit[a]!!
                circuits.getOrPut(key) { mutableListOf() } += b
                key
            }

            !aPreset && bPreset -> {
                val key = junctionBoxToCircuit[b]!!
                junctionBoxToCircuit[a] = junctionBoxToCircuit[b]!!
                circuits.getOrPut(key) { mutableListOf() } += a
                key
            }

            else -> {
                val key = junctionBoxToCircuit[a]!!
                val oldKey = junctionBoxToCircuit[b]!!
                if (oldKey == key) return key

                circuits[oldKey]!!.forEach {
                    junctionBoxToCircuit[it] = key
                    circuits.getOrPut(key) { mutableListOf() } += it
                }
                circuits -= oldKey
                key
            }
        }
    }

    override fun toString(): String =
        "Playground(junctionBoxes=$junctionBoxes, connectedCircuits=$circuits, totalCircuitCount=$circuitCount)"
}

fun String.parse(): Playground =
    Playground(trim().lines().map { it.trim().split(",").let { (x, y, z) -> JunctionBox(x.toInt(), y.toInt(), z.toInt()) } })

In [4]:
val exampleInput = """
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
"""

val exampleBoxes = exampleInput.parse()
exampleBoxes

Playground(junctionBoxes=[(162, 817, 812), (57, 618, 57), (906, 360, 560), (592, 479, 940), (352, 342, 300), (466, 668, 158), (542, 29, 236), (431, 825, 988), (739, 650, 466), (52, 470, 668), (216, 146, 977), (819, 987, 18), (117, 168, 530), (805, 96, 715), (346, 949, 466), (970, 615, 88), (941, 993, 340), (862, 61, 35), (984, 92, 344), (425, 690, 689)], connectedCircuits={}, totalCircuitCount=20)

In [5]:
exampleBoxes.carthesianProductByDistance.take(10).forEach {
    exampleBoxes.connect(it)
    DISPLAY("connecting: $it\nconnected circuits: ${exampleBoxes.circuits}\ntotal circuit count: ${exampleBoxes.circuitCount}\n")
}

connecting: ((162, 817, 812), (425, 690, 689))
connected circuits: {0=[(162, 817, 812), (425, 690, 689)]}
total circuit count: 19


connecting: ((162, 817, 812), (431, 825, 988))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988)]}
total circuit count: 18


connecting: ((906, 360, 560), (805, 96, 715))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988)], 1=[(906, 360, 560), (805, 96, 715)]}
total circuit count: 17


connecting: ((431, 825, 988), (425, 690, 689))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988)], 1=[(906, 360, 560), (805, 96, 715)]}
total circuit count: 17


connecting: ((862, 61, 35), (984, 92, 344))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988)], 1=[(906, 360, 560), (805, 96, 715)], 2=[(862, 61, 35), (984, 92, 344)]}
total circuit count: 16


connecting: ((52, 470, 668), (117, 168, 530))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988)], 1=[(906, 360, 560), (805, 96, 715)], 2=[(862, 61, 35), (984, 92, 344)], 3=[(52, 470, 668), (117, 168, 530)]}
total circuit count: 15


connecting: ((819, 987, 18), (941, 993, 340))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988)], 1=[(906, 360, 560), (805, 96, 715)], 2=[(862, 61, 35), (984, 92, 344)], 3=[(52, 470, 668), (117, 168, 530)], 4=[(819, 987, 18), (941, 993, 340)]}
total circuit count: 14


connecting: ((906, 360, 560), (739, 650, 466))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988)], 1=[(906, 360, 560), (805, 96, 715), (739, 650, 466)], 2=[(862, 61, 35), (984, 92, 344)], 3=[(52, 470, 668), (117, 168, 530)], 4=[(819, 987, 18), (941, 993, 340)]}
total circuit count: 13


connecting: ((346, 949, 466), (425, 690, 689))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988), (346, 949, 466)], 1=[(906, 360, 560), (805, 96, 715), (739, 650, 466)], 2=[(862, 61, 35), (984, 92, 344)], 3=[(52, 470, 668), (117, 168, 530)], 4=[(819, 987, 18), (941, 993, 340)]}
total circuit count: 12


connecting: ((906, 360, 560), (984, 92, 344))
connected circuits: {0=[(162, 817, 812), (425, 690, 689), (431, 825, 988), (346, 949, 466)], 1=[(906, 360, 560), (805, 96, 715), (739, 650, 466), (862, 61, 35), (984, 92, 344)], 3=[(52, 470, 668), (117, 168, 530)], 4=[(819, 987, 18), (941, 993, 340)]}
total circuit count: 11


In [6]:
exampleBoxes.circuits.values.map { it.size }.sortedDescending().take(3).reduce(Int::times)

40

In [7]:
val playground = aoc.input().parse()
playground.carthesianProductByDistance.take(1_000).forEach {
    playground.connect(it)
}

In [9]:
val result = playground.circuits.values.map { it.size }.sortedDescending().take(3).reduce(Int::times)
result

127551

In [10]:
// aoc.submitPartOne(result)

Your answer: 127551. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [11]:
aoc.viewPartTwo()

--- Part Two --- 
 The Elves were right; they definitely don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in one large circuit . 
 Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at 216,146,977 and 117,168,530 . The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes ( 216 and 117 ) produces 25272 . 
 Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit . What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?

In [12]:
val playground = aoc.input().parse()
lateinit var lastPair: Pair<JunctionBox, JunctionBox>
for (pair in playground.carthesianProductByDistance) {
    playground.connect(pair)
    if (playground.circuitCount == 1) {
        lastPair = pair
        break
    }
}
lastPair

((44966, 19938, 98403), (52200, 30637, 98911))

In [15]:
val (a, b) = lastPair
// aoc.submitPartTwo(a.x.toLong() * b.x.toLong())

Your answer: 2347225200. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 8! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .